In [4]:
from dotenv import load_dotenv
load_dotenv()

False

In [5]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, ChatHuggingFace, HuggingFaceEndpoint
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

Indexing (Document Ingestion)

In [6]:
from youtube_transcript_api._api import YouTubeTranscriptApi

video_id = "Gfr50f6ZBvo"

try:
    api = YouTubeTranscriptApi()
    transcript_list = api.fetch(video_id)
    
    transcript = " ".join([snippet.text for snippet in transcript_list])
 
    
except Exception as e:
    print(f"Error: {type(e).__name__}: {e}")

Indexing(Text Splitting)

In [7]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = splitter.create_documents([transcript])

In [8]:
len(chunks)

168

In [9]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [10]:
vector_store = FAISS.from_documents(chunks,embeddings)

In [11]:
vector_store.index_to_docstore_id

{0: 'c170958a-e0ff-4979-bd07-34ff6841187c',
 1: '0ff8e144-1710-42d7-a5c2-05a0ca84932d',
 2: 'ba668a32-cc81-4284-8dcb-a3a2f2d8ee40',
 3: '70b71f4c-fd03-43e0-a298-6ff94e8920e1',
 4: '79598da6-8e0d-4025-a8c9-85eff3470f5f',
 5: '176c64d3-7377-4f0e-a2e1-a2ccc44ba333',
 6: '71bc7f65-de03-4a94-afb3-fe2a91ce809b',
 7: 'f20b431a-9856-40d3-8e23-609a433766f8',
 8: '96728e48-5eb0-419e-915d-305bb12088c7',
 9: '59adda91-806b-4876-8bee-2dd0479dfd20',
 10: '3a7da429-66c7-4132-a90f-1f2cab7092a3',
 11: 'fe6cf04c-5c40-4e73-81c3-950f0082f326',
 12: 'fad98db3-6b0c-488d-b2a4-2f72486e5fdb',
 13: 'f15311b3-600f-4a33-a384-85bf9789acd7',
 14: '67ddabac-6063-47e0-a2e3-22d96e489645',
 15: 'ab942eb1-ba4a-4ab0-92c9-b01bafc9a99e',
 16: '6e44dc40-3cb9-4f7c-a067-7aceb8c9d3dc',
 17: 'df5ce2a0-e895-48ed-aee0-774ebf1ea003',
 18: '1e846526-233f-4863-b3fa-15a9b5abe027',
 19: 'f494fd42-9f5b-40c6-96d0-c90818873c4d',
 20: '962a77dc-e8cf-4692-9322-2422b95ac67f',
 21: '0d9379ee-0de7-47a9-ac56-6736d01e4730',
 22: '80ef7eda-2732-

Retrival

In [12]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs = {"k": 4})

In [13]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000028D27D19160>, search_kwargs={'k': 4})

In [14]:
result = retriever.invoke("What is deepmind?")
print(result[3].page_content)


ai we would call it now um people like minsky and and and patrick winston and you know all these characters right and used to debate a few of them and they used to think i was mad thinking about that some new advance could be done with learning systems and um i was actually pleased to hear that because at least you know you're on a unique track at that point right even if every all of your you know professors are telling you you're mad that's true and of course in industry uh you can we couldn't get you know as difficult to get two cents together uh and which is hard to imagine now as well given it's the biggest sort of buzzword in in vcs and and fundraising's easy and all these kind of things today so back in 2010 it was very difficult and what we the reason we started then and shane and i used to discuss um uh uh what were the sort of founding tenets of deep mind and it was very various things one was um algorithmic advances so deep learning you know jeff hinton and cohen just had


In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
llm = HuggingFaceEndpoint(
    repo_id="deepseek-ai/DeepSeek-V3.2",
    huggingfacehub_api_token="hf_your_new_token_here",
    task="text-generation",
    max_new_tokens=512, 
    temperature=0.7    
)
model = ChatHuggingFace(llm=llm)

In [23]:
prompt = PromptTemplate(
    template = """
    You are a helpful assistant.
    Answer only from the provided transcript context.
    If the context is insufficient, just say you don't know.
    
    {context}
    Question: {question}
    """,
    input_variables = ["context","question"]
)

In [24]:
question = "is the topic of aliens discussed in this video? if yes then what was discussed?"
retrieved_docs = retriever.invoke(question)

In [25]:
retrieved_docs

[Document(id='b07f3e77-7971-4839-ac0b-7a14bed373fc', metadata={}, page_content="space age we should have heard a cacophony of voices we should have joined that cacophony of voices and what we did we opened our ears and we heard nothing and many people who argue that there are aliens would say well we haven't really done exhaustive search yet and maybe we're looking in the wrong bands and and we've got the wrong devices and we wouldn't notice what an alien form was like to be so different to what we're used to but you know i'm not i don't really buy that that it shouldn't be as difficult as that like we i think we've searched enough there should be if it were everywhere if it was it should be everywhere we should see dyson's fears being put up sun's blinking in and out you know there should be a lot of evidence for those things and then there are other people argue well the sort of safari view of like well we're a primitive species still because we're not space faring yet and and and we

In [26]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"space age we should have heard a cacophony of voices we should have joined that cacophony of voices and what we did we opened our ears and we heard nothing and many people who argue that there are aliens would say well we haven't really done exhaustive search yet and maybe we're looking in the wrong bands and and we've got the wrong devices and we wouldn't notice what an alien form was like to be so different to what we're used to but you know i'm not i don't really buy that that it shouldn't be as difficult as that like we i think we've searched enough there should be if it were everywhere if it was it should be everywhere we should see dyson's fears being put up sun's blinking in and out you know there should be a lot of evidence for those things and then there are other people argue well the sort of safari view of like well we're a primitive species still because we're not space faring yet and and and we're you know there's some kind of globe like universal rule not to interfere\n\

In [27]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

Generation

In [28]:
answer = model.invoke(final_prompt)
print(answer.content)

Yes, the topic of aliens is discussed.

What was discussed:
- The expectation that, in the space age, we should have heard a "cacophony of voices" from alien civilizations, but we have heard nothing.
- Arguments that we might not have searched exhaustively or are looking in the wrong way are presented but countered with the idea that evidence like Dyson spheres or obvious cosmic engineering should be visible if alien life were common.
- The possibility that alien civilizations could be interacting with human consciousness is mentioned but questioned, as it seems unlikely all alien species would communicate uniformly in that way.
- It is suggested there should be a normal distribution of alien civilizations—some primitive, some aggressive, some philosophical—and not a uniform type.
- The "great filter" concept is discussed as a potential barrier preventing civilizations from becoming multi-planetary or reaching the stars, making it an important consideration for humanity's future.
- The

In [30]:
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [31]:
def format_docs(retrieved_docs):
    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return context_text

In [32]:
parallel_chain = RunnableParallel({
    "context": retriever | RunnableLambda(format_docs),
    "question": RunnablePassthrough()
})

In [34]:
parallel_chain.invoke("who is Demis?")

{'context': "to get world peace because there's also other corrupting things like wanting power over people and this kind of stuff which is not necessarily satisfied by by just abundance but i think it will help um and i think uh but i think ultimately ai is not going to be run by any one person or one organization i think it should belong to the world belong to humanity um and i think maybe many there'll be many ways this will happen and ultimately um everybody should have a say in that do you have advice for uh young people in high school and college maybe um if they're interested in ai or interested in having a big impact on the world what they should do to have a career they can be proud of her to have a life they can be proud of i love giving talks to the next generation what i say to them is actually two things i i think the most important things to learn about and to find out about when you're when you're young is what are your true passions is first of all there's two things on

In [35]:
parser = StrOutputParser()

In [37]:
main_chain = parallel_chain | prompt | model | parser

In [38]:
main_chain.invoke("Can you summarize this video")

'Based on the provided transcript, this video appears to be a conversation discussing the need for a simpler, more fundamental explanation of physics that goes beyond the current standard model. It touches on topics like consciousness, life, and gravity, and includes reflections on intelligence, explanation, and historical moments in computing and AI, such as early chess programs and Deep Blue.'